# Azure AI Gateway - Policy Fragment Management

![Architecture](../../images/fragment-policies.png)

## Overview

This notebook demonstrates advanced APIM policy management using **Policy Fragments**, **Feature Flags**, and **Named Values** for dynamic, reusable policy composition.

### Key Benefits

- **Reusable Components**: Define policies once, use across multiple APIs
- **Feature Flags**: Enable/disable policies without redeployment
- **Centralized Configuration**: Manage policy parameters via Named Values
- **Easy Testing**: Test policies independently and in combinations
- **Version Control Friendly**: Small, focused XML fragments
- **CI/CD Ready**: Export/import configurations for automated deployments

### Architecture

```
API Request → APIM Gateway → Master Policy (conditional fragment includes)
                                ↓
                    Feature Flags (Named Values)
                                ↓
        ┌───────────────────────┴───────────────────────┐
        ↓                       ↓                       ↓
   Fragment 1              Fragment 2              Fragment N
   (if enabled)            (if enabled)            (if enabled)
```

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/)
- [VS Code](https://code.visualstudio.com/) with [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and authenticated
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- Python packages: `azure-mgmt-apimanagement`, `azure-identity`, `requests`, `pandas`, `matplotlib`

### Fragments Included

1. **Token Metrics** - Emit token consumption to Application Insights
2. **Load Balancing** - Backend pool with priority and retry logic
3. **Token Rate Limiting** - Enforce TPM limits per subscription
4. **Private Connectivity** - Managed identity authentication
5. **Semantic Caching** - Redis-based prompt caching
6. **Circuit Breaker** - Fault tolerance and resilience

Run `Run All` to execute sequentially, or execute step by step.

## Section 0: Setup & Configuration

Initialize the notebook environment, import libraries, and configure Azure credentials.

In [ ]:
# Cell 2: Imports and Environment Setup
import os
import sys
import json
import subprocess
import time
import requests
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# Add shared utilities to path
sys.path.insert(1, '../../shared')
import utils

# Set up matplotlib defaults
mpl.rcParams['figure.figsize'] = [15, 7]

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ Environment initialized successfully")
print(f"⏰ Notebook started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Cell 3: Configuration Class
@dataclass
class FragmentPolicyConfig:
    """Configuration for fragment-based policy management."""
    
    # Azure Configuration
    subscription_id: str = ""
    resource_group: str = "lab-fragment-policies"
    location: str = "uksouth"
    
    # APIM Service Configuration
    apim_service: str = ""
    apim_sku: str = "Basicv2"
    api_id: str = "azure-openai-api"
    api_path: str = "inference"
    api_version: str = "2025-03-01-preview"
    
    # AI Services Configuration
    aiservices_config: List[Dict] = field(default_factory=lambda: [
        {"name": "foundry1", "location": "uksouth"}
    ])
    
    # Model Configuration
    models_config: List[Dict] = field(default_factory=lambda: [
        {"name": "gpt-4o-mini", "publisher": "OpenAI", "version": "2024-07-18", 
         "sku": "GlobalStandard", "capacity": 20}
    ])
    
    # Feature Flags (default states)
    features: Dict[str, bool] = field(default_factory=lambda: {
        "token-metrics": True,
        "load-balancing": True,
        "token-ratelimit": False,  # Off by default for testing
        "private-connectivity": False,  # Requires private endpoints
        "caching": False,  # Requires Redis cache
        "circuit-breaker": True
    })
    
    # Configuration Values (Named Value defaults)
    config_values: Dict[str, str] = field(default_factory=lambda: {
        "token-metrics-namespace": "openai",
        "lb-backend-pool-id": "openai-backend-pool",
        "lb-retry-count": "2",
        "token-ratelimit-tpm": "100",
        "private-backend-id": "openai-private-backend",
        "cache-duration-seconds": "300",
        "cache-score-threshold": "0.8",
        "cache-embeddings-backend": "openai-embeddings-backend",
        "cache-embeddings-auth": "subscription-key",
        "circuit-error-threshold": "5",
        "circuit-timeout-seconds": "60",
        "circuit-window-seconds": "120"
    })
    
    def to_dict(self) -> Dict:
        """Convert configuration to dictionary."""
        return asdict(self)
    
    def save(self, filepath: str) -> None:
        """Save configuration to JSON file."""
        Path(filepath).write_text(json.dumps(self.to_dict(), indent=2))
        print(f"✅ Configuration saved to: {filepath}")
    
    @classmethod
    def load(cls, filepath: str) -> 'FragmentPolicyConfig':
        """Load configuration from JSON file."""
        data = json.loads(Path(filepath).read_text())
        return cls(**data)

# Initialize configuration
config = FragmentPolicyConfig()
print("✅ Configuration class initialized")
print(f"📁 Resource Group: {config.resource_group}")
print(f"🌍 Location: {config.location}")
print(f"🔧 APIM SKU: {config.apim_sku}")

In [ ]:
# Cell 4: Fragment Definitions
FRAGMENTS = {
    "fragment-token-metrics": {
        "description": "Azure OpenAI token metrics emission to Application Insights",
        "xml_template": "fragments/token-metrics.xml",
        "applies_to": ["inbound"],
        "feature_flag": "feature-token-metrics-enabled",
        "order": 1
    },
    "fragment-load-balancing": {
        "description": "Backend pool load balancing with retry logic",
        "xml_template": "fragments/load-balancing.xml",
        "applies_to": ["inbound", "backend"],
        "feature_flag": "feature-load-balancing-enabled",
        "order": 2
    },
    "fragment-token-ratelimit": {
        "description": "Token-based rate limiting (TPM enforcement)",
        "xml_template": "fragments/token-ratelimit.xml",
        "applies_to": ["inbound"],
        "feature_flag": "feature-token-ratelimit-enabled",
        "order": 3
    },
    "fragment-private-connectivity": {
        "description": "Managed identity authentication for private endpoints",
        "xml_template": "fragments/private-connectivity.xml",
        "applies_to": ["inbound"],
        "feature_flag": "feature-private-connectivity-enabled",
        "order": 4
    },
    "fragment-caching": {
        "description": "Semantic caching with Azure Redis",
        "xml_template": "fragments/caching.xml",
        "applies_to": ["inbound", "outbound"],
        "feature_flag": "feature-caching-enabled",
        "order": 5
    },
    "fragment-circuit-breaker": {
        "description": "Circuit breaker pattern for fault tolerance",
        "xml_template": "fragments/circuit-breaker.xml",
        "applies_to": ["inbound", "on-error"],
        "feature_flag": "feature-circuit-breaker-enabled",
        "order": 6
    }
}

print(f"✅ Loaded {len(FRAGMENTS)} fragment definitions:")
for frag_id, frag_info in sorted(FRAGMENTS.items(), key=lambda x: x[1]['order']):
    applies = ", ".join(frag_info['applies_to'])
    print(f"   {frag_info['order']}. {frag_id}: {frag_info['description']}")
    print(f"      Applies to: {applies}")

In [ ]:
# Cell 5: Helper Function - Run Azure CLI Command
def run_az_command(command: str, success_msg: str = "", error_msg: str = "", 
                   return_json: bool = True) -> Tuple[bool, Any]:
    """
    Execute Azure CLI command and return results.
    
    Args:
        command: Azure CLI command to execute
        success_msg: Message to display on success
        error_msg: Message to display on error
        return_json: Parse output as JSON
    
    Returns:
        Tuple of (success: bool, output: dict/str)
    """
    try:
        print(f"⚙️  Running: {command[:100]}..." if len(command) > 100 else f"⚙️  Running: {command}")
        
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            timeout=300  # 5 minute timeout
        )
        
        if result.returncode == 0:
            if success_msg:
                print(f"✅ {success_msg}")
            
            if return_json and result.stdout.strip():
                try:
                    return True, json.loads(result.stdout)
                except json.JSONDecodeError:
                    return True, result.stdout.strip()
            return True, result.stdout.strip()
        else:
            error = error_msg or "Command failed"
            print(f"❌ {error}")
            print(f"   Error: {result.stderr.strip()}")
            return False, result.stderr.strip()
            
    except subprocess.TimeoutExpired:
        print(f"❌ Command timed out after 5 minutes")
        return False, "Timeout"
    except Exception as e:
        print(f"❌ Exception: {str(e)}")
        return False, str(e)

print("✅ Helper function 'run_az_command' defined")

In [ ]:
# Cell 6: Helper Function - Deploy Fragment
def deploy_fragment(fragment_id: str, xml_content: str, description: str) -> bool:
    """
    Deploy a policy fragment to APIM.
    
    Args:
        fragment_id: Unique identifier for the fragment
        xml_content: XML policy content
        description: Human-readable description
    
    Returns:
        True if deployment succeeded, False otherwise
    """
    # Save XML to temporary file
    temp_file = Path(f"/tmp/{fragment_id}.xml")
    temp_file.write_text(xml_content)
    
    # Deploy using Azure CLI
    command = f"""az apim api policy-fragment create \
        --resource-group {config.resource_group} \
        --service-name {config.apim_service} \
        --policy-fragment-id {fragment_id} \
        --description "{description}" \
        --value-path {temp_file}"""
    
    success, _ = run_az_command(
        command,
        success_msg=f"Fragment '{fragment_id}' deployed",
        error_msg=f"Failed to deploy fragment '{fragment_id}'"
    )
    
    # Clean up temp file
    temp_file.unlink(missing_ok=True)
    
    return success

print("✅ Helper function 'deploy_fragment' defined")

In [ ]:
# Cell 7: Helper Function - Create Named Value
def create_named_value(name: str, value: str, secret: bool = False) -> bool:
    """
    Create or update a Named Value in APIM.
    
    Args:
        name: Named value identifier
        value: Value to store
        secret: Whether to store as secret (encrypted)
    
    Returns:
        True if creation/update succeeded
    """
    secret_flag = "--secret true" if secret else ""
    
    command = f"""az apim nv create-or-update \
        --resource-group {config.resource_group} \
        --service-name {config.apim_service} \
        --named-value-id {name} \
        --value "{value}" \
        {secret_flag}"""
    
    success, _ = run_az_command(
        command,
        success_msg=f"Named value '{name}' = '{value[:20]}...' created" if len(value) > 20 else f"Named value '{name}' = '{value}' created",
        error_msg=f"Failed to create named value '{name}'"
    )
    
    return success

print("✅ Helper function 'create_named_value' defined")

In [ ]:
# Cell 8: Helper Function - Toggle Feature
def toggle_feature(feature_name: str, enabled: bool) -> bool:
    """
    Toggle a feature flag on/off.
    
    Args:
        feature_name: Feature name (without 'feature-' prefix)
        enabled: True to enable, False to disable
    
    Returns:
        True if toggle succeeded
    """
    named_value_name = f"feature-{feature_name}-enabled"
    value = "true" if enabled else "false"
    
    success = create_named_value(named_value_name, value, secret=False)
    
    if success:
        # Update local config
        config.features[feature_name] = enabled
        status = "✅ ENABLED" if enabled else "❌ DISABLED"
        print(f"   {status} - {feature_name}")
    
    return success

print("✅ Helper function 'toggle_feature' defined")

In [ ]:
# Cell 9: Helper Function - Apply Master Policy
def apply_master_policy(api_id: str, policy_xml: str) -> bool:
    """
    Apply master policy to an API.
    
    Args:
        api_id: API identifier
        policy_xml: Complete policy XML with fragment includes
    
    Returns:
        True if policy applied successfully
    """
    # Save policy to temporary file
    temp_file = Path("/tmp/master-policy.xml")
    temp_file.write_text(policy_xml)
    
    command = f"""az apim api policy create \
        --resource-group {config.resource_group} \
        --service-name {config.apim_service} \
        --api-id {api_id} \
        --xml-path {temp_file}"""
    
    success, _ = run_az_command(
        command,
        success_msg=f"Master policy applied to API '{api_id}'",
        error_msg=f"Failed to apply policy to API '{api_id}'"
    )
    
    # Clean up
    temp_file.unlink(missing_ok=True)
    
    return success

print("✅ Helper function 'apply_master_policy' defined")

In [ ]:
# Cell 10: Helper Function - List Fragments
def list_fragments() -> List[Dict]:
    """
    List all policy fragments in APIM.
    
    Returns:
        List of fragment dictionaries
    """
    command = f"""az apim api policy-fragment list \
        --resource-group {config.resource_group} \
        --service-name {config.apim_service}"""
    
    success, output = run_az_command(command, success_msg="Retrieved fragment list")
    
    if success and isinstance(output, list):
        return output
    return []

print("✅ Helper function 'list_fragments' defined")
print("\n✅ All helper functions initialized successfully")
print("   - run_az_command")
print("   - deploy_fragment")
print("   - create_named_value")
print("   - toggle_feature")
print("   - apply_master_policy")
print("   - list_fragments")

## Section 1: Fragment Deployment

Deploy all policy fragments to Azure API Management.

### Initialize Azure Environment

Verify Azure CLI connection and retrieve subscription details.

In [ ]:
# Cell 13: Verify Azure CLI and Subscription
success, account_info = run_az_command(
    "az account show",
    success_msg="Retrieved Azure account information"
)

if success:
    config.subscription_id = account_info['id']
    tenant_id = account_info['tenantId']
    current_user = account_info['user']['name']
    
    print(f"\n👤 User: {current_user}")
    print(f"🆔 Tenant: {tenant_id}")
    print(f"📋 Subscription: {config.subscription_id}")
else:
    print("❌ Failed to retrieve Azure account. Please run 'az login' first.")

In [ ]:
# Cell 14: Create Resource Group
print(f"📦 Creating resource group: {config.resource_group}")
print(f"🌍 Location: {config.location}")

command = f"""az group create \
    --name {config.resource_group} \
    --location {config.location} \
    --tags source=fragment-policies lab=ai-gateway"""

success, _ = run_az_command(
    command,
    success_msg=f"Resource group '{config.resource_group}' created",
    error_msg="Resource group already exists or creation failed"
)

### Load Fragment XML Files

Read all fragment XML files from the `fragments/` directory.

In [ ]:
# Cell 16: Load Fragment XML Files
print("📂 Loading fragment XML files...\n")

fragment_contents = {}

for fragment_id, fragment_info in FRAGMENTS.items():
    xml_path = Path(fragment_info["xml_template"])
    
    if xml_path.exists():
        xml_content = xml_path.read_text()
        fragment_contents[fragment_id] = xml_content
        
        # Display fragment info
        print(f"✅ {fragment_id}")
        print(f"   File: {xml_path}")
        print(f"   Size: {len(xml_content)} bytes")
        print(f"   Applies to: {', '.join(fragment_info['applies_to'])}")
        print()
    else:
        print(f"❌ {fragment_id}: File not found at {xml_path}")
        print()

print(f"📊 Loaded {len(fragment_contents)}/{len(FRAGMENTS)} fragment files")

In [ ]:
# Cell 17: Display Fragment XML (Token Metrics Example)
print("📄 Example Fragment: Token Metrics\n")
print("="*80)

if "fragment-token-metrics" in fragment_contents:
    print(fragment_contents["fragment-token-metrics"])
else:
    print("Fragment not loaded")

print("="*80)

### Deploy Fragments to APIM

Note: This step requires an existing APIM service. Update `config.apim_service` with your APIM service name.

In [ ]:
# Cell 19: Deploy All Fragments to APIM
# IMPORTANT: Set your APIM service name
config.apim_service = "YOUR-APIM-SERVICE-NAME"  # TODO: Update this!

if config.apim_service == "YOUR-APIM-SERVICE-NAME":
    print("⚠️  WARNING: Please set config.apim_service to your APIM service name")
    print("   Example: config.apim_service = 'my-apim-service'")
else:
    print(f"🚀 Deploying fragments to APIM: {config.apim_service}\n")
    
    deployment_results = []
    
    # Sort fragments by deployment order
    sorted_fragments = sorted(FRAGMENTS.items(), key=lambda x: x[1]['order'])
    
    for fragment_id, fragment_info in sorted_fragments:
        if fragment_id in fragment_contents:
            print(f"\n📤 Deploying: {fragment_id}")
            print(f"   Description: {fragment_info['description']}")
            
            success = deploy_fragment(
                fragment_id,
                fragment_contents[fragment_id],
                fragment_info['description']
            )
            
            deployment_results.append({
                "fragment": fragment_id,
                "status": "✅ SUCCESS" if success else "❌ FAILED"
            })
            
            # Brief pause between deployments
            time.sleep(1)
    
    # Summary
    print("\n" + "="*80)
    print("📊 DEPLOYMENT SUMMARY")
    print("="*80)
    
    df = pd.DataFrame(deployment_results)
    print(df.to_string(index=False))
    
    success_count = sum(1 for r in deployment_results if "SUCCESS" in r['status'])
    print(f"\n✅ Successfully deployed: {success_count}/{len(deployment_results)}")

In [ ]:
# Cell 20: Verify Fragment Deployment
print("🔍 Verifying fragment deployment...\n")

if config.apim_service != "YOUR-APIM-SERVICE-NAME":
    deployed_fragments = list_fragments()
    
    print(f"📊 Total fragments in APIM: {len(deployed_fragments)}\n")
    
    if deployed_fragments:
        # Create verification table
        verification_data = []
        
        for fragment_id in FRAGMENTS.keys():
            # Check if fragment exists in APIM
            found = any(f.get('name') == fragment_id for f in deployed_fragments)
            
            verification_data.append({
                "Fragment": fragment_id,
                "Expected": "✅",
                "Deployed": "✅" if found else "❌",
                "Status": "OK" if found else "MISSING"
            })
        
        df = pd.DataFrame(verification_data)
        print(df.to_string(index=False))
        
        # Summary
        deployed_count = sum(1 for d in verification_data if d['Status'] == 'OK')
        print(f"\n✅ Verified: {deployed_count}/{len(FRAGMENTS)} fragments deployed")
    else:
        print("⚠️  No fragments found in APIM")
else:
    print("⚠️  Skipping verification - APIM service name not configured")

## Section 2: Feature Flags & Configuration

Deploy feature flags and configuration values as APIM Named Values.